In [2130]:
import numpy as np
import pandas as pd
import os

# PART 1 of Allurion case study, by Juan Lopez
#### Read me:

##### Welcome to the part one of this case study. My main objective in this part is not only prove that I am capable of cleaning and processing data by myself, but to prove that I can write production-ready quality scripts that can actually be deployed in cloud environments and used for automatize takes for future use. It doesn't  matter if the inputs contain different data and contains more data than it this example, this code should take care of it and can be modified easily to solve bugs and to deal with unexpected data.

###### Side note: Yes, this is displayed in a jupyter notebook so you can visualize and test things by yourself easily. But if this code was designed in such way that if it is saved as a python script and executed it will do its job ;)


### Loading data (read me)
Loading the data. In this project we are going name the sheets 'INPUT_CUSTOMER_DATABASE', 'INPUT_CLINIC_LOCATOR', 'INPUT_DISTRIBUTOR_SELL_OUT', and 'PARAM_TABLES' by 'sheet_0', 'sheet_1', 'sheet_2', and 'sheet_3' respectively

In [2131]:
""" Loading data """
def load_data():
    filepath="/Users/juansmacbook/Desktop/Allurionproject/Part_1_Case_Study_Data_Inputs_v2.0.xlsx"
    sheet_0=pd.read_excel(filepath, header=14, usecols="C:O", sheet_name=0)
    sheet_1=pd.read_excel(filepath, header=13, usecols="B:G", sheet_name=1)
    sheet_2=pd.read_excel(filepath, header=13, usecols="B:G", sheet_name=2)
    sheet_3=pd.read_excel(filepath, header=11, usecols="C:E", sheet_name=3)
    print("Data has been loaded sucessfully ;)")
    return sheet_0, sheet_1, sheet_2, sheet_3
sheet_0, sheet_1, sheet_2, sheet_3 = load_data()

Data has been loaded sucessfully ;)


## 1) Transforming doctor specialities: Mapping the specialities in 'sheet_0' according to the parameters table on PARAM_TABLES

In this first part we are going to do the cleaning of specialities in 'sheet_0' and 'sheet_3' (PARAM_TABLES) so later we can use the info in 'sheet_3' to every speciality in 'sheet_0' into their right name




Here what we are going to do is automatically cleaning the param table, and then we are using it to modify all the professions in the 'sheet_0'.
Note that 'sheet_1' and 'sheet_2' don't display any information of specialities about new possible professionals (at least not correctly enough so we can use them)


In [2132]:
"""Quick function to delete repeated spaces and lower case all letters, we are going to apply this function to the 'sheet_3' (PARAM_TABLE) and the specialities in 'sheet_0' so they are nice and clean to be modified later """
def delete_spaces_and_lowercase(sentence):
    sentence=sentence.lower()
    sentence=" ".join(sentence.split())
    return  sentence

"""Lets apply the delete_spaces_and_lowercase() function to sheet1 and sheet3 as we already mentioned"""

sheet_3["HCP Initial Specialty"]=sheet_3["HCP Initial Specialty"].apply(lambda x: delete_spaces_and_lowercase(x))
for column in ["HCP specialty\n1","HCP specialty\n2","HCP specialty\n3","HCP specialty\n4","HCP specialty\n5"]:
    sheet_0[column]=sheet_0[column].apply(lambda x: delete_spaces_and_lowercase(x))

"""Let's look at the professions on the sheet0 that are not on the params table, If there are any we will manually add them"""
unique_professions=np.unique(np.concatenate((
    sheet_0["HCP specialty\n1"].unique(),
    sheet_0["HCP specialty\n2"].unique(),
    sheet_0["HCP specialty\n3"].unique(),
    sheet_0["HCP specialty\n4"].unique(),
    sheet_0["HCP specialty\n5"].unique()), axis=0))
unique_professions=pd.Series(unique_professions, name="unique_professions")

"""As you can see all specialities on the INPUT_CUSTOMER_DATABASE appear in our param table :)"""
"""I actually designed the functions so they will throw an exception with a message if the condition above is not satisfied"""


print(unique_professions.apply(lambda x: x in sheet_3["HCP Initial Specialty"].to_numpy()).value_counts())
print("For our fortune, all the cleaned specialities on INPUT_CUSTOMER now appear on our PARAM_TABLE, there is no need to manually add any :)")
print("Note for production: if we received any false value it means that we must update the PARAM_TABLE and add the corresponding speciality names, otherwise the function below will rise an exception! ")



"""Since the parameter are nice and clean, the following function will take care of mapping them to the correct label """

def map_speciality_name(speciality):

    row=sheet_3[sheet_3["HCP Initial Specialty"]==speciality][0:1]
    special_char=["-", "None","Other"]

    if (row["Final Label\nHCP Specialty"].values in special_char) and (row["Contact Role"].values in special_char ):
        correct_label=row["HCP Initial Specialty"].values
    elif (row["Final Label\nHCP Specialty"].values not in special_char) and (row["Contact Role"].values in special_char ):
        correct_label=row["Final Label\nHCP Specialty"].values
    elif (row["Final Label\nHCP Specialty"].values in special_char) and (row["Contact Role"].values not in special_char ):
        correct_label=row["Contact Role"].values

    elif (row["Final Label\nHCP Specialty"].values not in special_char) and (row["Contact Role"].values not in special_char ):
        correct_label=row["Final Label\nHCP Specialty"].values
    else:
        raise Exception("Hey! The speciality that you input in the function map_speciality_name() is not in the parameters list, or it has to valid labels")
    return correct_label[0]

#test function for testing the function map_speciality_name()
#for item in sheet_3["HCP Initial Specialty"]:
#    print(map_speciality_name(item))


True    36
Name: unique_professions, dtype: int64
For our fortune, all the cleaned specialities on INPUT_CUSTOMER now appear on our PARAM_TABLE, there is no need to manually add any :)
Note for production: if we received any false value it means that we must update the PARAM_TABLE and add the corresponding speciality names, otherwise the function below will rise an exception! 


### Lets map specialities on sheet_1 using the map_speciality_name() function
Now we are going to modify every speciality in the 'sheet_0' using the function we defined above

In [2133]:
""" We are mapping every speciality according to param table"""

sheet_0["HCP specialty\n1"]=sheet_0["HCP specialty\n1"].apply(lambda x: map_speciality_name(x))
sheet_0["HCP specialty\n2"]=sheet_0["HCP specialty\n2"].apply(lambda x: map_speciality_name(x))
sheet_0["HCP specialty\n3"]=sheet_0["HCP specialty\n3"].apply(lambda x: map_speciality_name(x))
sheet_0["HCP specialty\n4"]=sheet_0["HCP specialty\n4"].apply(lambda x: map_speciality_name(x))
sheet_0["HCP specialty\n5"]=sheet_0["HCP specialty\n5"].apply(lambda x: map_speciality_name(x))


## PART 2) Filling new doctor names in 'sheet_0' from 'sheet_1' and 'sheet_2'

We have a lot of data from sheet 1 and 2 that is not appearing in 'sheet_0'. Therefore, we are going to add it so the resulting names are NEVER repeated.
To do this, we must do extensive data cleaning, so we can distinguish between the same doctor name written in a lot of different ways.

For now, we are going to start with some data preprocessing basics

In [2134]:
""" We rename sheet 1 and sheet 2 column to 'Customer unique ID' so they have a matching column with sheet 0. This is really important since we are going to malke make a joins from 'sheet_1' and 'sheet_2' into 'sheet_0' according to their unique ID identifier"""

sheet_1.rename(columns = {'Customer Unique ID':'Customer unique ID'}, inplace = True)
sheet_2.rename(columns = {'CUSTOMER UNIQUE ID':'Customer unique ID'}, inplace = True)

""" We are also renaming other columns form the sheet 2 so it is easier to locate them when we join all the dataframes"""
sheet_2.rename(columns = {'CLINIC NAME':'Clinic Name Italy'}, inplace = True)
sheet_2.rename(columns = {'HCP\nname 1':'HCP Name 1 Italy'}, inplace = True)
sheet_2.rename(columns = {'HCP\nname 2':'HCP Name 2 Italy'}, inplace = True)
sheet_2.rename(columns = {'HCP\nname 3':'HCP Name 3 Italy'}, inplace = True)

Let's joins onto the 'sheet_0' using the unique ID identifier

In [2135]:
"""Lets Join the Sheet 1 on Sheet 0, we will use this sheet for operations"""
sheet_joined=sheet_0.join(sheet_1[["Customer unique ID","Clinic Name","HCP Name"]].set_index('Customer unique ID'), on='Customer unique ID')
sheet_joined=sheet_joined.join(sheet_2[["Customer unique ID","Clinic Name Italy",'HCP Name 1 Italy', 'HCP Name 2 Italy', 'HCP Name 3 Italy']].set_index('Customer unique ID'), on='Customer unique ID')

"""Let's replace NaN resulting from the join with '-' """
for columns in sheet_joined.columns:
    sheet_joined[columns]=sheet_joined[columns].fillna(value='-')


In [2136]:
""" Let's reorganize the name of the columns and call only the ones that are interesting for us"""
sheet_joined=sheet_joined[["Customer unique ID","Clinic Name","Clinic Name Italy","HCP Name",'HCP Name 1 Italy','HCP Name 2 Italy','HCP Name 3 Italy',"HCP Name\n1","HCP Name\n2","HCP Name\n3","HCP Name\n4","HCP Name\n5" ]]


### Data cleaning pipeline
In this part we are going to be creating functions and applying them into 'sheet_0', 'sheet_0' and 'sheet_0'. This ensures we are treating the data in the same 'format' and strucrure so we don't misinterpret data with equal names just because of special characters, accents, capital letters, or becasue the position of the first and the last name are flipped

In [2137]:
"""Defining a function to eliminate special characters that may do our life harder """
def delete_special_chars(string):
    special_chars=['Dr ','med. ', 'Dr. ', 'Drs ', 'med. ', 'DR ', 'Dott ', 'Dott.ssa ', 'Prof ', 'prof ', 'pr ', 'dott ', 'met.','dott. ','drssa ','dr. ', ' dott. ','dott.']
    for char in special_chars:
        string=string.replace(char, '')
    return string

"""We are defining a function that will let us extract and separate multiple names that are into stuck in the same string separated by commas or weird characters"""

import re
def separate_into_lists(string):
    return re.split('; |, | & | et | / ', string)
import unidecode
def delete_accents(string):
    return unidecode.unidecode(string)

"""Lets apply the previous functions that we defined and some others that are really helpful"""

for column in ["HCP Name",'HCP Name 1 Italy','HCP Name 2 Italy','HCP Name 3 Italy',"HCP Name\n1","HCP Name\n2","HCP Name\n3","HCP Name\n4","HCP Name\n5"]:
    sheet_joined[column]=sheet_joined[column].apply(lambda x: delete_special_chars(x))
    sheet_joined[column]=sheet_joined[column].apply(lambda x: delete_accents(x))
    sheet_joined[column]=sheet_joined[column].apply(lambda x: x.lower())
    sheet_joined[column]=sheet_joined[column].apply(lambda x: separate_into_lists(x) if x!='-' else '-')
#sheet_joined[sheet_joined.columns[3:]]

## Adding the new and non-repeated clean names to 'sheet_0'
This is the fun part, here we have to do a lot of processing so we can now exactly what names are we going to add. the names of sheet_0, 'sheet_1' and 'sheet_2' are already as cleaned as I could. However, there are a lot names that are written in different form and we have to detect that in order to do a really good job.

For example, let's say that we have two doctor names for the same professional, perhps 'dr. lopez juan' and 'j. lopez', our algorithm must detect that this names are actually the same.

Next, we have to manipulate the data in all the sheets, so we know exactly what names are repeated and how we are going to inject them into the sheet_0 without repeating. To do so, we have to quite a long and challenging process, which took a lot of time, so I made sure my job was done correctly.

In [2138]:
"""The following function will help us compare two strings, if the two strings are the same, or similar in some way, the function will return True"""
""" This function is extremely useful and we will be used in the final process"""


def compare_names(string1,string2): #Recieves two names and return whether the professionals are the same
    string1_to_return=string1
    string2_to_return=string2
    string1=string1.split(' ')
    string2=string2.split(' ')
    if ' ' in string1:
        string1.remove('')
    if '' in string2:
        string2.remove('')
    return np.in1d(np.array(string1),np.array(string2)).any(), string1_to_return, string2_to_return #Returns a triple
compare_names(" ", " test string ")


(True, ' ', ' test string ')

### Transformation pipeline

This is the fun part. Since our data at this point have been through a process of cleaning and standardization. The next step is going to be creating a function that will take the whole dataframe form 'sheet_0' and will do a series of manipulations, transformations and mapping so the output is nice and clean.

The next function will take care of that. Inside it will do various subprocesses that are really important. I will try to explain them as good as possible

In [2139]:

"""df0 to df3 refers to columns from sheet_1 and sheet_2. df4 to df8 are the entries from sheet_0"""
def merge_names(df0,df1,df2,df3,df4,df5,df6,df7,df8):

#""" Group the inputs form sheet_0 and name them as 'old_entries', and group new incoming data from sheet_1 and sheet_2 together '"""
    new_entries = [df0,df1,df2,df3]
    old_entries = [df4,df5,df6,df7,df8]

#"""Unlist the names on sheet_0 since they are encapsulated in a List"""
    slots_to_add=[] #This is how we are calling the 'slots' on sheet_0 that we are going to fill
    for item in old_entries:
        if item=='-': slots_to_add.append(item)
        else: slots_to_add.append(item[0])

#"""Joining all the new entries in lists; those form she sheet 1 and from the sheet2. New list will be called HCP and will be the one used to be modified until it contains the correct info that we will have to be injected to the 'slots_to_add'"""

    HCP=[]
    for item in new_entries:
        if item!='-': HCP=HCP+item;
    if HCP==[]: HCP=['-']

#"""Let's look at the list of names that we have to add, and transform so every list contains unique names """
    HCP_iter=HCP
    if len(HCP)>1:
        for item in HCP:
            i=0
            statements=[]
            while i<len(HCP_iter):
                statement,_,_=compare_names(item,HCP_iter[i])
                statements.append(statement)
                i+=1
            if sum(statements)>1:
                HCP_iter.remove(item)
#"""NOTE: From now, we are not going to keep using HCP as the data that have to be injected, but a copy of it called HCP_iter"""

#"""Since we already now that the data in HCP_iter is unique, We are going to remove the names that are already appearing in 'sheet_0'"""
    for i in range(len(slots_to_add)):
        j=0
        while j<len(HCP_iter):
            found, _, _ = compare_names(HCP_iter[j],slots_to_add[i])
            """We select the longest string"""
            if found==True:
                if len(HCP_iter[j])>len(slots_to_add[i]): longest_string=HCP_iter[j];
                else: longest_string=slots_to_add[i];
                slots_to_add[i]=longest_string
                HCP_iter.remove(HCP_iter[j])
            j+=1

#"""For now, the HCP_iter lists contains only unique names that have to be applied in the sheet_0 """

#"""Now let's inject the professionals into the right columns on sheet_0 """
    if HCP_iter!=[]:
        for i in range(len(slots_to_add)):
            if slots_to_add[i] != '-':
                pass
            else:
                if HCP_iter != []:
                    slots_to_add[i] = HCP_iter.pop(0)


#"""defining outputs"""
    output= pd.Series(slots_to_add)

    return output



### Putting all together

We are calling the pipeline function that we just defined, and we are applying it into the actual 'sheet_0', so the original Excel file is transformed into its  correct version!

In [2140]:
"""Let's pass the data trough our pipeline!"""
col0,col1,col2,col3,col4,col5,col6,col7,col8 = sheet_joined.columns[3:]
result=sheet_joined[sheet_joined.columns[3:]].apply(lambda x: merge_names(x[col0], x[col1], x[col2], x[col3], x[col4], x[col5], x[col6], x[col7], x[col8]), axis=1)
sheet_0["HCP Name\n1"]=result[0]
sheet_0["HCP Name\n2"]=result[1]
sheet_0["HCP Name\n3"]=result[2]
sheet_0["HCP Name\n4"]=result[3]
sheet_0["HCP Name\n5"]=result[4]


### Now the data of sheet_0 (INPUT_CUSTOMER_DATABASE) have been modified and ready to be saved as an Excel file :)

Let's take a look at it and save it as an Excel file

In [2141]:
sheet_0.to_excel("output_part1.xlsx")
sheet_0

,Customer unique ID,Account Name,HCP Name\n1,HCP specialty\n1,HCP Name\n2,HCP specialty\n2,HCP Name\n3,HCP specialty\n3,HCP Name\n4,HCP specialty\n4,HCP Name\n5,HCP specialty\n5,Clinic Locator ID
0,CH000001,Dr Nikolaus Linde,nikolaus linde,Aesthetic physician,-,-,-,-,-,-,-,-,-
1,CH000002,Espace Chirurgie - Bern,christof buser,-,andre witschi,-,-,-,-,-,-,-,3812
2,CH000003,Studio Medico Switzerland,andrea morri,-,-,-,-,-,-,-,-,-,3814
3,CH000004,Clinique Rive Gauche,neda barouti,dermatologist,cathy,Patient Manager / Coordinator General,-,-,-,-,-,-,4180
4,CH000005,Dr Dimitrios Samaras,dimitrios samaras,internist,-,-,-,-,-,-,-,-,4181
...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,SE000006,Cavalio - Malmö,-,-,-,-,-,-,-,-,-,-,4237
187,SE000007,Cavalio - Göteborg,-,-,-,-,-,-,-,-,-,-,4238
188,SE000008,Gastroklinik - Malmö,haytham al-tayar,-,ali dalfi,-,-,-,-,-,-,-,3818
189,SE000009,Gastroklinik - Gothenburg,haytham al-tayar,-,ali dalfi,-,-,-,-,-,-,-,3817


In [2142]:
ls

Case_Study_Instructions_v3.0.pptx
Part_1_Case_Study_Data_Inputs_v2.0.xlsx
Part_2_Case_Study_Data_Inputs_v2.0.xlsb
main.ipynb
output_part1.xlsx
venv/
~$Part_1_Case_Study_Data_Inputs_v2.0.xlsx
